In [1]:
# Background: We obtained demographic information extracted from the EDW 
# based on all names and dates of birth recorded in MUSE (the ECG database). 
# Here we combine all the files Annabel and Brianna sent over together.
# The end goal is to obtain a PatientID for each relevant patient in MUSE, 
# so that we can extract outcomes stored in the EDW (accomplished in step02 notebook).
import pandas as pd

# Load all demographics files we have access to
demographics_files= []
for i in range(5):
    demographics_files.append(pd.read_csv('../data_EDW_AF/afib_dems_a_part' + str(i+1) + '.csv', dtype='str'))
first_file = pd.concat(demographics_files)
first_file.drop(columns=['lastnm_edw', 'firstnm_edw', 'mrn_edw1', 'mrn_edw2', 'EDWPatientID'], inplace=True)

second_file = pd.read_csv('../data_EDW_AF/ECGPositive_EDWinfo.csv', dtype='str')
second_file['PatientRaceFinal'].fillna('nan', inplace=True)
second_file['race_ethn_5'] = second_file['PatientRaceFinal'].map({'White': 'white', 
                                                                    'Declined or Unavailable': 'all_other',
                                                                    'Hispanic or Latino': 'hisp_latin',
                                                                    'Black of African American': 'black',
                                                                    'Native American or Pacific Islandar': 'all_other',
                                                                    'nan': 'all_other',
                                                                    'Other': 'all_other'})
second_file.drop(columns=['MRN', 'EDWPatientID'], inplace=True)


third_file = []
col_names = pd.read_csv('../data_EDW_AF/museleft_dems_part1.csv').columns
for part in range(3):
    if part == 0:
        third_file.append(pd.read_csv('../data_EDW_AF/museleft_dems_part' + str(part+1) + '.csv', dtype='str'))
    else:
        third_file.append(pd.read_csv('../data_EDW_AF/museleft_dems_part' + str(part+1) + '.csv', names=col_names, dtype='str'))
third_file = pd.concat(third_file)

third_file.drop(columns=['lastnm_edw', 'firstnm_edw', 'mrn_edw1', 'mrn_edw2'], inplace=True)



In [67]:
cols = ['PatientFirstName', 'PatientLastName', 'DOB']

for df in [first_file, second_file, third_file]:
    df['PatientFirstName'] = df['PatientFirstName'].str.lower()
    df['PatientLastName'] = df['PatientLastName'].str.lower()

    df['DOB'] = pd.to_datetime(df['DOB']).dt.floor('D')
    df['DOB'] = df['DOB'].dt.strftime('%m-%d-%Y')
    df['name_dob'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

muse_edw_map = pd.concat([first_file, second_file, third_file])
muse_edw_map.to_csv('../outputs_intermediate/muse_edw_map.csv')
